<a href="https://colab.research.google.com/github/CalvHobbes/ecomm_ai/blob/main/sqlcoder8b_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "sqlcoder[transformers]"

In [2]:
!wget -O ecomm_db_postgres.sql https://raw.githubusercontent.com/CalvHobbes/ecomm_ai/fe18fd15b4a41a01919a3f1cbb5a4ee5b85305e0/ecomm_db_postgres.sql
!wget -O insert_sample_data.sql https://raw.githubusercontent.com/CalvHobbes/ecomm_ai/fe18fd15b4a41a01919a3f1cbb5a4ee5b85305e0/insert_sample_data.sql


--2025-02-10 09:18:53--  https://raw.githubusercontent.com/CalvHobbes/ecomm_ai/fe18fd15b4a41a01919a3f1cbb5a4ee5b85305e0/ecomm_db_postgres.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2065 (2.0K) [text/plain]
Saving to: ‘ecomm_db_postgres.sql’

ecomm_db_postgres.s 100%[===================>]   2.02K  --.-KB/s    in 0s      

2025-02-10 09:18:53 (40.6 MB/s) - ‘ecomm_db_postgres.sql’ saved [2065/2065]

--2025-02-10 09:18:53--  https://raw.githubusercontent.com/CalvHobbes/ecomm_ai/fe18fd15b4a41a01919a3f1cbb5a4ee5b85305e0/insert_sample_data.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... co

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import argparse

def generate_prompt(question, prompt_file="sqlcoder-prompt.md", metadata_file="ecomm_db_postgres.sql"):
    with open(prompt_file, "r") as f:
        prompt = f.read()

    with open(metadata_file, "r") as f:
        table_metadata_string = f.read()

    prompt = prompt.format(
        user_question=question, table_metadata_string=table_metadata_string
    )
    return prompt


In [2]:
!pip install -U bitsandbytes

In [3]:
from transformers import BitsAndBytesConfig

def get_tokenizer_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=True,
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
        quantization_config=quantization_config,
    )
    return tokenizer, model

def run_inference(question, prompt_file="sqlcoder-prompt.md", metadata_file="ecomm_db_postgres.sql"):
    prompt = generate_prompt(question, prompt_file, metadata_file)

    # make sure the model stops generating at triple ticks
    # eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]
    eos_token_id = tokenizer.eos_token_id
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=300,
        do_sample=False,
        return_full_text=False, # added return_full_text parameter to prevent splitting issues with prompt
        num_beams=5, # do beam search with 5 beams for high quality results
    )
    generated_query = (
        pipe(
            prompt,
            num_return_sequences=1,
            eos_token_id=eos_token_id,
            pad_token_id=eos_token_id,
        )[0]["generated_text"]
        .split(";")[0]
        .split("```")[0]
        .strip()
        + ";"
    )
    return generated_query

tokenizer, model = get_tokenizer_model("defog/llama-3-sqlcoder-8b")

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [7]:
question = 'how many orders completed in the last week?'
run_inference(question)

Device set to use cuda:0


"SELECT COUNT(o.id) FROM orders o WHERE o.status = 'delivered' AND o.created_at >= (CURRENT_DATE - interval '7 days');"